In [1]:
import logging

import numpy

from cicada.communicator import SocketCommunicator
from cicada.shamir import ShamirBasicProtocol
import cicada.shamir

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.logging.Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocol(communicator=communicator, threshold=2)
    
    secret = numpy.array(42) if communicator.rank == 0 else None
    share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())

    revealed = protocol.encoder.decode(protocol.reveal(share))
    log.info(f"Player {communicator.rank} revealed: {revealed}")
    
SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 revealed: 42.0
INFO:root:Player 1 revealed: 42.0
INFO:root:Player 2 revealed: 42.0


With the program working, we can simulate a fault by killing player 0 before the secret is revealed:

In [2]:
import os
import signal

def main(communicator):
    log = cicada.logging.Logger(logging.getLogger(), communicator=communicator)
    protocol = cicada.shamir.ShamirBasicProtocol(communicator=communicator, threshold=2)
    
    secret = numpy.array(42) if communicator.rank == 0 else None
    share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())

    if communicator.rank == 0:
        os.kill(os.getpid(), signal.SIGKILL)

    revealed = protocol.encoder.decode(protocol.reveal(share))
    log.info(f"Player {communicator.rank} revealed: {revealed}")
    
SocketCommunicator.run(world_size=3, fn=main);

ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 1 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 812, in launch
    result = fn(communicator, *args, **kwargs)
  File "/var/folders/tl/h2xygzzn1154jzjn_n01x860001l4n/T/ipykernel_82323/144164982.py", line 14, in main
    revealed = protocol.encoder.decode(protocol.reveal(share))
  File "/Users/tshead/src/cicada-mpc/cicada/shamir.py", line 387, in reveal
    received_shares = self.communicator.gatherv(src=src, value=share, dst=recipient)
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 571, in gatherv
    values = [self._wait_next_payload(src=rank, tag=Tags.GATHERV) for rank in src]
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 571, in <listco

In [3]:
import os
import signal

def main(communicator):
    try:
        log = cicada.logging.Logger(logging.getLogger(), communicator=communicator)
        protocol = cicada.shamir.ShamirBasicProtocol(communicator=communicator, threshold=2)

        secret = numpy.array(42) if communicator.rank == 0 else None
        share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())

        if communicator.rank == 0:
            os.kill(os.getpid(), signal.SIGKILL)

        revealed = protocol.encoder.decode(protocol.reveal(share))
        log.info(f"Player {communicator.rank} revealed: {revealed}")
        
    except Exception as e:
        log.sync = False
        log.info(f"Player {communicator.rank} exception: {e}")
    
SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 2 exception: Tag GATHERV from player 0 timed-out after 5s
INFO:root:Player 1 exception: Tag GATHERV from player 0 timed-out after 5s


In [4]:
def main(communicator):
    log = cicada.logging.Logger(logging.getLogger(), communicator=communicator)
    protocol = cicada.shamir.ShamirBasicProtocol(communicator=communicator, threshold=2)

    try:
        secret = numpy.array(42) if communicator.rank == 0 else None
        share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())

        if communicator.rank == 0:
            os.kill(os.getpid(), signal.SIGKILL)

        revealed = protocol.encoder.decode(protocol.reveal(share))
        log.info(f"Player {communicator.rank} revealed: {revealed}")
        
    except Exception as e:
        log.sync = False
        log.info(f"Player {communicator.rank} exception: {e}")
        
        communicator, oldranks = communicator.shrink(name="new")
        with communicator:
            log = cicada.logging.Logger(logging.getLogger(), communicator=communicator)
            protocol = cicada.shamir.ShamirBasicProtocol(communicator=communicator, indices=protocol.indices[oldranks], threshold=2)

            revealed = protocol.encoder.decode(protocol.reveal(share))
            log.info(f"Player {communicator.rank} revealed: {revealed}")
        
SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 1 exception: Tag GATHERV from player 0 timed-out after 5s
INFO:root:Player 2 exception: Tag GATHERV from player 0 timed-out after 5s
INFO:root:Player 0 revealed: 42.0
INFO:root:Player 1 revealed: 42.0
